In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import cv2
from skimage import filters
from skimage.feature import blob_log
from skimage.measure import label, regionprops
import os
import pandas as pd
import math

## Detecció d'axons

In [2]:
folder = "S_blanca"
imgs_gray = []
im_names = os.listdir(folder)
for filename in im_names:
    imgs_gray.append(plt.imread(os.path.join(folder,filename)))

#### Blob detector

In [13]:
true_pos = 0
false_pos = 0
false_neg = 0

for i in range(len(imgs_gray)):
    # CÀLCUL DELS AXONS
    blobs_log = blob_log(imgs_gray[i], min_sigma=3, max_sigma=13, num_sigma=6, threshold=0.3)


    # AVALUACIÓ DEL RESULTAT
    db = pd.read_csv('res_quant/heatmap/'+im_names[i][:2]+'-quant.csv')
    db = db[db['Area'] > 6]

    db_aux = db.copy()
    blobs_aux = blobs_log
    correctes = np.empty((0,3))

    while((blobs_aux.shape[0] > 0) and (db_aux.shape[0] > 0)):
        min_distancies = np.empty((0,3))
        for j in range(blobs_aux.shape[0]):
            blob = blobs_aux[j,:]
            db_aux['Distancies'] = db_aux.apply(lambda x: math.dist([blob[1], blob[0]], [x['X'], x['Y']]), axis=1)
            minim = db_aux['Distancies'].min()
            index = db_aux['Distancies'].idxmin()
            min_distancies = np.append(min_distancies, np.array([[j, index, minim]]), axis=0)
        
        indexs1 = []
        indexs2 = []
        min_distancies = min_distancies[min_distancies[:,2].argsort()]
        if (min_distancies[0,2] > 5):
            break
        for j in range(min_distancies.shape[0]):
            if (min_distancies[0,2] > 5):
                break
            if min_distancies[j,1] not in indexs2:
                indexs1.append(int(min_distancies[j,0]))
                indexs2.append(min_distancies[j,1])
                correctes = np.append(correctes, np.array([blobs_aux[int(min_distancies[j,0]), :]]), axis=0)
        blobs_aux = np.delete(blobs_aux, indexs1, axis=0)
        db_aux.drop(indexs2, inplace=True)

    print("Imatge ", im_names[i][:-5])
    print("Axons obtinguts: ", str(blobs_log.shape[0]))
    print("Axons de la solució: ", str(db.shape[0]))
    print("True Positives: ", correctes.shape[0])
    print("False Positives:", blobs_aux.shape[0])
    print("False Negatives:", db_aux.shape[0])
    print("Precision: ", round(correctes.shape[0] / (correctes.shape[0]+blobs_aux.shape[0]), 3))
    print("Recall: ", round(correctes.shape[0] / (correctes.shape[0]+db_aux.shape[0]), 3), "\n")

    true_pos = true_pos + correctes.shape[0]
    false_pos = false_pos + blobs_aux.shape[0]
    false_neg = false_neg + db_aux.shape[0]


    # GUARDAR LA IMATGE RESULTANT
    im_axons = np.empty([imgs_gray[i].shape[0], imgs_gray[i].shape[1], 3])
    im_axons[:,:,2] = imgs_gray[i]
    for axo in correctes:
        y, x, r = axo
        im_axons = cv2.circle(im_axons, (int(x), int(y)), int(r), (255, 0, 0), 2)
    for axo in blobs_aux:
        y, x, r = axo
        im_axons = cv2.circle(im_axons, (int(x), int(y)), int(r), (0, 255, 0), 2)      
    for j in range(db_aux.shape[0]):
        axo = db_aux.iloc[j]
        im_axons = cv2.circle(im_axons, (int(axo['X']), int(axo['Y'])), int(np.sqrt(axo['Area']/np.pi)), (255, 255, 255), 2)
    cv2.imwrite("detect_axons/resultats/blob_detector/"+im_names[i][:-5]+".jpg", im_axons)


print("Precision general: ", round(true_pos / (true_pos+false_pos), 3))
print("Recall general: ", round(true_pos / (true_pos+false_neg), 3))

Imatge  C1-BJZ5
Axons obtinguts:  430
Axons de la solució:  429
True Positives:  374
False Positives: 56
False Negatives: 55
Precision:  0.87
Recall:  0.872 

Imatge  C2-BJZ5
Axons obtinguts:  299
Axons de la solució:  428
True Positives:  259
False Positives: 40
False Negatives: 169
Precision:  0.866
Recall:  0.605 

Imatge  C3-BJZ5
Axons obtinguts:  325
Axons de la solució:  256
True Positives:  234
False Positives: 91
False Negatives: 22
Precision:  0.72
Recall:  0.914 

Imatge  C4-BJZ5
Axons obtinguts:  205
Axons de la solució:  256
True Positives:  153
False Positives: 52
False Negatives: 103
Precision:  0.746
Recall:  0.598 

Imatge  C5-BJZ5
Axons obtinguts:  242
Axons de la solució:  213
True Positives:  181
False Positives: 61
False Negatives: 32
Precision:  0.748
Recall:  0.85 

Imatge  C6-BJZ5
Axons obtinguts:  167
Axons de la solució:  227
True Positives:  139
False Positives: 28
False Negatives: 88
Precision:  0.832
Recall:  0.612 

Imatge  C7-BJZ5
Axons obtinguts:  216
Axo

#### Threshold + Opening

In [24]:
true_pos = 0
false_pos = 0
false_neg = 0

for i in range(len(imgs_gray)):
    # CÀLCUL DELS AXONS
    im_punts = np.where(imgs_gray[i]>165, 255, 0)
    kernel = np.ones((3, 3), np.uint8)
    im_open = cv2.morphologyEx(im_punts.astype(np.uint8), cv2.MORPH_OPEN, kernel)

    label_img = label(im_open)
    regions = regionprops(label_img)


    # AVALUACIÓ DEL RESULTAT
    db = pd.read_csv('res_quant/heatmap/'+im_names[i][:2]+'-quant.csv')
    db = db[db['Area'] > 6]

    db_aux = db.copy()
    regions_aux = regions
    correctes = []

    while((len(regions_aux) > 0) and (db_aux.shape[0] > 0)):
        min_distancies = np.empty((0,3))
        for j in range(len(regions_aux)):
            axo = regions_aux[j]
            db_aux['Distancies'] = db_aux.apply(lambda x: math.dist([axo.centroid[1], axo.centroid[0]], [x['X'], x['Y']]), axis=1)
            minim = db_aux['Distancies'].min()
            index = db_aux['Distancies'].idxmin()
            min_distancies = np.append(min_distancies, np.array([[j, index, minim]]), axis=0)
        
        indexs1 = []
        indexs2 = []
        min_distancies = min_distancies[min_distancies[:,2].argsort()]
        if (min_distancies[0,2] > 5):
            break
        for j in range(min_distancies.shape[0]):
            if (min_distancies[0,2] > 5):
                break
            if min_distancies[j,1] not in indexs2:
                indexs1.append(min_distancies[j,0])
                indexs2.append(min_distancies[j,1])
                correctes.append(regions_aux[int(min_distancies[j,0])])
        regions_aux = [x for j, x in enumerate(regions_aux) if j not in indexs1]
        db_aux.drop(indexs2, inplace=True)

    print("Imatge ", im_names[i][:-5])
    print("Axons obtinguts: ", str(len(regions)))
    print("Axons de la solució: ", str(db.shape[0]))
    print("True Positives: ", len(correctes))
    print("False Positives:", len(regions_aux))
    print("False Negatives:", db_aux.shape[0])
    print("Precision: ", round(len(correctes) / (len(correctes)+len(regions_aux)), 3))
    print("Recall: ", round(len(correctes) / (len(correctes)+db_aux.shape[0]), 3), "\n")

    true_pos = true_pos + len(correctes)
    false_pos = false_pos + len(regions_aux)
    false_neg = false_neg + db_aux.shape[0]


    # GUARDAR LA IMATGE RESULTANT
    im_axons = np.empty([imgs_gray[i].shape[0], imgs_gray[i].shape[1], 3])
    im_axons[:,:,2] = imgs_gray[i]
    for axo in correctes:
        im_axons = cv2.circle(im_axons, (int(axo.centroid[1]), int(axo.centroid[0])), int(axo.equivalent_diameter_area/2), (255, 0, 0), 2)
    for axo in regions_aux:
        im_axons = cv2.circle(im_axons, (int(axo.centroid[1]), int(axo.centroid[0])), int(axo.equivalent_diameter_area/2), (0, 255, 0), 2)
    for j in range(db_aux.shape[0]):
        axo = db_aux.iloc[j]
        im_axons = cv2.circle(im_axons, (int(axo['X']), int(axo['Y'])), int(np.sqrt(axo['Area']/np.pi)), (255, 255, 255), 2)
    cv2.imwrite("detect_axons/resultats/threshold/"+im_names[i][:-5]+".jpg", im_axons)
        

print("Precision general: ", round(true_pos / (true_pos+false_pos), 3))
print("Recall general: ", round(true_pos / (true_pos+false_neg), 3))

Imatge  C1-BJZ5
Axons obtinguts:  430
Axons de la solució:  429
True Positives:  386
False Positives: 44
False Negatives: 43
Precision:  0.898
Recall:  0.9 

Imatge  C2-BJZ5
Axons obtinguts:  309
Axons de la solució:  428
True Positives:  274
False Positives: 35
False Negatives: 154
Precision:  0.887
Recall:  0.64 

Imatge  C3-BJZ5
Axons obtinguts:  293
Axons de la solució:  256
True Positives:  240
False Positives: 53
False Negatives: 16
Precision:  0.819
Recall:  0.938 

Imatge  C4-BJZ5
Axons obtinguts:  201
Axons de la solució:  256
True Positives:  158
False Positives: 43
False Negatives: 98
Precision:  0.786
Recall:  0.617 

Imatge  C5-BJZ5
Axons obtinguts:  227
Axons de la solució:  213
True Positives:  182
False Positives: 45
False Negatives: 31
Precision:  0.802
Recall:  0.854 

Imatge  C6-BJZ5
Axons obtinguts:  167
Axons de la solució:  227
True Positives:  143
False Positives: 24
False Negatives: 84
Precision:  0.856
Recall:  0.63 

Imatge  C7-BJZ5
Axons obtinguts:  224
Axons

#### Opening Residue

In [27]:
true_pos = 0
false_pos = 0
false_neg = 0

for i in range(len(imgs_gray)):
    # CÀLCUL DELS AXONS
    kernel = np.ones((35, 35), np.uint8)
    im_open = cv2.morphologyEx(imgs_gray[i], cv2.MORPH_OPEN, kernel)
    im_open_res = imgs_gray[i]- im_open
    im_punts = np.where(im_open_res>150, 255, 0)
    kernel = np.ones((3, 3), np.uint8)
    im_punts_open = cv2.morphologyEx(im_punts.astype(np.uint8), cv2.MORPH_OPEN, kernel)

    label_img = label(im_punts_open)
    regions = regionprops(label_img)


    # AVALUACIÓ DEL RESULTAT
    db = pd.read_csv('res_quant/heatmap/'+im_names[i][:2]+'-quant.csv')
    db = db[db['Area'] > 6]

    db_aux = db.copy()
    regions_aux = regions
    correctes = []

    while((len(regions_aux) > 0) and (db_aux.shape[0] > 0)):
        min_distancies = np.empty((0,3))
        for j in range(len(regions_aux)):
            axo = regions_aux[j]
            db_aux['Distancies'] = db_aux.apply(lambda x: math.dist([axo.centroid[1], axo.centroid[0]], [x['X'], x['Y']]), axis=1)
            minim = db_aux['Distancies'].min()
            index = db_aux['Distancies'].idxmin()
            min_distancies = np.append(min_distancies, np.array([[j, index, minim]]), axis=0)
        
        indexs1 = []
        indexs2 = []
        min_distancies = min_distancies[min_distancies[:,2].argsort()]
        if (min_distancies[0,2] > 5):
            break
        for j in range(min_distancies.shape[0]):
            if (min_distancies[0,2] > 5):
                break
            if min_distancies[j,1] not in indexs2:
                indexs1.append(min_distancies[j,0])
                indexs2.append(min_distancies[j,1])
                correctes.append(regions_aux[int(min_distancies[j,0])])
        regions_aux = [x for j, x in enumerate(regions_aux) if j not in indexs1]
        db_aux.drop(indexs2, inplace=True)

    print("Imatge ", im_names[i][:-5])
    print("Axons obtinguts: ", str(len(regions)))
    print("Axons de la solució: ", str(db.shape[0]))
    print("True Positives: ", len(correctes))
    print("False Positives:", len(regions_aux))
    print("False Negatives:", db_aux.shape[0])
    print("Precision: ", round(len(correctes) / (len(correctes)+len(regions_aux)), 3))
    print("Recall: ", round(len(correctes) / (len(correctes)+db_aux.shape[0]), 3), "\n")

    true_pos = true_pos + len(correctes)
    false_pos = false_pos + len(regions_aux)
    false_neg = false_neg + db_aux.shape[0]


    # GUARDAR LA IMATGE RESULTANT
    im_axons = np.empty([imgs_gray[i].shape[0], imgs_gray[i].shape[1], 3])
    im_axons[:,:,2] = imgs_gray[i]
    for axo in correctes:
        im_axons = cv2.circle(im_axons, (int(axo.centroid[1]), int(axo.centroid[0])), int(axo.equivalent_diameter_area/2), (255, 0, 0), 2)
    for axo in regions_aux:
        im_axons = cv2.circle(im_axons, (int(axo.centroid[1]), int(axo.centroid[0])), int(axo.equivalent_diameter_area/2), (0, 255, 0), 2)
    for j in range(db_aux.shape[0]):
        axo = db_aux.iloc[j]
        im_axons = cv2.circle(im_axons, (int(axo['X']), int(axo['Y'])), int(np.sqrt(axo['Area']/np.pi)), (255, 255, 255), 2)
    cv2.imwrite("detect_axons/resultats/opening_residue/"+im_names[i][:-5]+".jpg", im_axons)
    

print("Precision general: ", round(true_pos / (true_pos+false_pos), 3))
print("Recall general: ", round(true_pos / (true_pos+false_neg), 3))

Imatge  C1-BJZ5
Axons obtinguts:  411
Axons de la solució:  429
True Positives:  374
False Positives: 37
False Negatives: 55
Precision:  0.91
Recall:  0.872 

Imatge  C2-BJZ5
Axons obtinguts:  298
Axons de la solució:  428
True Positives:  266
False Positives: 32
False Negatives: 162
Precision:  0.893
Recall:  0.621 

Imatge  C3-BJZ5
Axons obtinguts:  294
Axons de la solució:  256
True Positives:  235
False Positives: 59
False Negatives: 21
Precision:  0.799
Recall:  0.918 

Imatge  C4-BJZ5
Axons obtinguts:  195
Axons de la solució:  256
True Positives:  154
False Positives: 41
False Negatives: 102
Precision:  0.79
Recall:  0.602 

Imatge  C5-BJZ5
Axons obtinguts:  216
Axons de la solució:  213
True Positives:  174
False Positives: 42
False Negatives: 39
Precision:  0.806
Recall:  0.817 

Imatge  C6-BJZ5
Axons obtinguts:  163
Axons de la solució:  227
True Positives:  140
False Positives: 23
False Negatives: 87
Precision:  0.859
Recall:  0.617 

Imatge  C7-BJZ5
Axons obtinguts:  214
Ax

#### Imatges bones

In [13]:
for ind, im in enumerate(imgs):
    if (ind==8):
        db = pd.read_csv('res_quant/heatmap/T1-quant.csv')
    else:
        db = pd.read_csv('res_quant/heatmap/C'+str(ind+1)+'-quant.csv')
    im_blobs = np.empty([im[:,:,0].shape[0], im[:,:,0].shape[1], 3])
    im_blobs[:,:,2] = im[:,:,0]
    c_stars = 0
    for i in range(db.shape[0]):
        axo = db.iloc[i]
        im_blobs = cv2.circle(im_blobs, (int(axo['X']), int(axo['Y'])), int(np.sqrt(axo['Area']/np.pi)), (255, 0, 0), 2)
        c_stars += 1
    print("Imatge "+str(ind+1)+ "- Número d'axons: " + str(c_stars))
    if (ind==8):
        cv2.imwrite("detect_axons/bones/T1.jpg", im_blobs)
    else:
        cv2.imwrite("detect_axons/bones/C"+str(ind+1)+".jpg", im_blobs)

Imatge 1- Número d'axons: 539
Imatge 2- Número d'axons: 540
Imatge 3- Número d'axons: 319
Imatge 4- Número d'axons: 328
Imatge 5- Número d'axons: 253
Imatge 6- Número d'axons: 309
Imatge 7- Número d'axons: 292
Imatge 8- Número d'axons: 222
Imatge 9- Número d'axons: 157
